In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import math
import sklearn.metrics
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import itertools
import copy
import os
import shutil
import urllib.request
import csv
import pandas as pd
from scipy.stats import bootstrap
import statistics
from PIL import Image
import warnings
from datetime import datetime

from sklearn.model_selection import RepeatedKFold
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve, auc

In [8]:
aps_train = pd.read_csv('../data/aps_failure_training_set.csv',header = 14)
aps_test = pd.read_csv('../data/aps_failure_test_set.csv',header = 14)

In [10]:
aps_test

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,neg,60,0,20,12,0,0,0,0,0,...,1098,138,412,654,78,88,0,0,0,0
1,neg,82,0,68,40,0,0,0,0,0,...,1068,276,1620,116,86,462,0,0,0,0
2,neg,66002,2,212,112,0,0,0,0,0,...,495076,380368,440134,269556,1315022,153680,516,0,0,0
3,neg,59816,na,1010,936,0,0,0,0,0,...,540820,243270,483302,485332,431376,210074,281662,3232,0,0
4,neg,1814,na,156,140,0,0,0,0,0,...,7646,4144,18466,49782,3176,482,76,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15995,neg,81852,na,2130706432,892,0,0,0,0,0,...,632658,273242,510354,373918,349840,317840,960024,25566,0,0
15996,neg,18,0,52,46,8,26,0,0,0,...,266,44,46,14,2,0,0,0,0,0
15997,neg,79636,na,1670,1518,0,0,0,0,0,...,806832,449962,778826,581558,375498,222866,358934,19548,0,0
15998,neg,110,na,36,32,0,0,0,0,0,...,588,210,180,544,1004,1338,74,0,0,0


In [3]:
#conda install -c pytorch faiss-cpu


In [15]:
import faiss
from joblib import Parallel, delayed

class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k

    def fit(self, X, y):
        self.index = faiss.IndexFlatL2(X.shape[1])
        self.index.add(X.astype(np.float32))
        self.y = y

    def predict(self, X):
        distances, indices = self.index.search(X.astype(np.float32), k=self.k)
        votes = self.y[indices]
        predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return predictions

def impute_col(X, y, mask, i, knn):
    col = X.iloc[:, i]
    if col.isna().sum() > 0:
        col_missing = col[mask[:, i]]
        col_missing = col_missing.values.reshape(-1, 1)
        col_not_missing = col[~mask[:, i]].values
        indices = knn.index.search(col_missing.astype(np.float32), k=5)[1]
        values = y[indices].mean(axis=1)
        col_imputed = np.concatenate([col_not_missing, values])
        X.iloc[:, i] = col_imputed
    return X

def parallel_impute(X, y, knn):
    mask = np.isnan(X)
    results = Parallel(n_jobs=-1, backend="multiprocessing")(delayed(impute_col)(X, y, mask, i, knn) for i in range(X.shape[1]))
    return pd.concat(results, axis=1)

In [16]:
# Extract the features and target
X_train = aps_train.drop('class', axis=1)
y_train = aps_train['class']

# Replace missing values with NaN
X_train[X_train == 'na'] = np.nan

# Convert the data to float
X_train = X_train.astype(np.float32)

# Create a FaissKNeighbors object and fit the data
knn = FaissKNeighbors(k=5)
knn.fit(X_train.values, y_train.values)

# Generate a mask for the missing values
mask = np.isnan(X_train)

# Impute the missing values using the FaissKNeighbors object
X_imputed = copy.deepcopy(X_train)

for i in range(X_imputed.shape[1]):
    col = X_imputed.iloc[:, i]
    if col.isna().sum() > 0:
        print(mask.shape)
        col_missing = col[mask[:, i]]
        col_missing = col_missing.values.reshape(-1, 1)
        col_not_missing = col[~mask[:, i]].values
        indices = knn.index.search(col_missing.astype(np.float32), k=5)[1]
        values = y_train[indices].mean(axis=1)
        col_imputed = np.concatenate([col_not_missing, values])
        X_imputed.iloc[:, i] = col_imputed

# Print the imputed data
print(X_imputed)

(60000, 170)


InvalidIndexError: (slice(None, None, None), 1)

In [28]:
mask = np.isnan(X_train.values)

col_missing = col[mask[:, i]]
col_missing = col_missing.values.reshape(-1, 1)
col_missing

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]], dtype=float32)